# Data Preprocessing

## Import Library

In [61]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

## 데이터 로드

In [62]:
path = '../data/time-based-split/1_train.csv'
filename = os.path.basename(path)
df = pd.read_csv(path)

## Binary 라벨링 (악성 트래픽: 1, 정상: 0)

In [63]:
# 컬럼명 공백 제거
df.columns = df.columns.str.strip()

df['Label_binary'] = df['Label'].apply(lambda x: 0 if x == 'BENIGN' else 1)
df['Label_binary'].value_counts()

Label_binary
0    154675
1    128399
Name: count, dtype: int64

## 불필요한 열 제거
- 식별자(Identifiers) 또는 고유값
- 목표값과 직접적인 관련이 있는 열 (Leakage Feature)
- 상수 값 (Constant Features)
- 결측치가 너무 많은 열
- 중복 정보 or 높은 상관관계를 가지는 열
- 텍스트, 로그, 비정형 필드

### 상수값 데이터 제거

In [64]:
low_variance_cols = [col for col in df.columns if df[col].nunique() <= 1]
low_variance_cols

['Bwd PSH Flags',
 'Fwd URG Flags',
 'Bwd URG Flags',
 'CWE Flag Count',
 'Fwd Avg Bytes/Bulk',
 'Fwd Avg Packets/Bulk',
 'Fwd Avg Bulk Rate',
 'Bwd Avg Bytes/Bulk',
 'Bwd Avg Packets/Bulk',
 'Bwd Avg Bulk Rate']

In [65]:
df = df.drop(columns=low_variance_cols)

### 결측 비율이 90% 이상인 데이터 제거

In [66]:
missing_cols = df.columns[df.isnull().mean() > 0.9]
missing_cols

Index([], dtype='object')

In [67]:
df = df.drop(columns=missing_cols)

### 목표값과 직접적인 관련이 있는 데이터 제거

In [68]:
leakage_cols = ['Label']
leakage_cols

['Label']

In [69]:
df = df.drop(columns=leakage_cols)

### 높은 상관관계 데이터 제거 (상관계수 0.95 이상)

**상관계수 0.95 이상 쌍 확인**

In [70]:
# 수치형 데이터만 추출
df_numeric = df.select_dtypes(include=[np.number])

# 상관계수 행렬 계산 (절댓값 기준)
corr_matrix = df_numeric.corr().abs()

# 상삼각 행렬 추출 (중복 제거용)
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# 상관계수 0.95 이상인 쌍 추출
high_corr_pairs = []

for col in upper_tri.columns:
    for row in upper_tri.index:
        corr_value = upper_tri.loc[row, col]
        if pd.notnull(corr_value) and corr_value >= 0.95:
            high_corr_pairs.append((row, col, round(corr_value, 4)))

# 내림차순 정렬
high_corr_pairs.sort(key=lambda x: x[2], reverse=True)

# 결과 출력
for a, b, corr in high_corr_pairs:
    print(f"{a:30} <--> {b:30} : {corr}")

Fwd PSH Flags                  <--> SYN Flag Count                 : 1.0
RST Flag Count                 <--> ECE Flag Count                 : 1.0
Fwd Packet Length Mean         <--> Avg Fwd Segment Size           : 1.0
Bwd Packet Length Mean         <--> Avg Bwd Segment Size           : 1.0
Fwd Header Length              <--> Fwd Header Length.1            : 1.0
Total Fwd Packets              <--> Subflow Fwd Packets            : 1.0
Total Length of Fwd Packets    <--> Subflow Fwd Bytes              : 1.0
Total Backward Packets         <--> Subflow Bwd Packets            : 1.0
Total Length of Bwd Packets    <--> Subflow Bwd Bytes              : 1.0
Packet Length Mean             <--> Average Packet Size            : 0.9993
Flow Duration                  <--> Fwd IAT Total                  : 0.9973
Flow IAT Max                   <--> Idle Max                       : 0.9959
Flow IAT Max                   <--> Fwd IAT Max                    : 0.9953
Bwd Packet Length Max          <--> Bwd

**제거할 feature 추출**

In [71]:
features_to_drop = set()
features_to_keep = set()
keywords = ['.1', 'Subflow', 'Avg', 'Segment', 'Bytes', 'Bulk']
already_dropped = set()

for col in upper_tri.columns:
    for row in upper_tri.index:
        corr_value = upper_tri.loc[row, col]
        if pd.notnull(corr_value) and corr_value >= 0.95:
            # 이미 제거된 컬럼은 무시
            if row in features_to_drop or col in features_to_drop:
                continue
            
            # 1. 파생/복제 이름 기준
            if any(kw in row for kw in keywords) and not any(kw in col for kw in keywords):
                drop_feature = row
                keep_feature = col
            elif any(kw in col for kw in keywords) and not any(kw in row for kw in keywords):
                drop_feature = col
                keep_feature = row
            
            # 2. 결측치가 많은 쪽 제거
            elif df[row].isnull().sum() > df[col].isnull().sum():
                drop_feature = row
                keep_feature = col
            elif df[col].isnull().sum() > df[row].isnull().sum():
                drop_feature = col
                keep_feature = row
            
            # 3. 고유값이 적은 쪽 제거
            elif df[row].nunique() < df[col].nunique():
                drop_feature = row
                keep_feature = col
            elif df[col].nunique() < df[row].nunique():
                drop_feature = col
                keep_feature = row
            
            # 4. 나머지는 col 제거(순서상 일관성 위해)
            else:
                drop_feature = col
                keep_feature = row
            features_to_drop.add(drop_feature)
            features_to_keep.add(keep_feature)

features_to_drop

{'Active Min',
 'Average Packet Size',
 'Avg Bwd Segment Size',
 'Avg Fwd Segment Size',
 'Bwd Header Length',
 'Bwd Packet Length Max',
 'Bwd Packet Length Mean',
 'ECE Flag Count',
 'Flow IAT Max',
 'Fwd Header Length.1',
 'Fwd IAT Max',
 'Fwd IAT Total',
 'Fwd Packet Length Max',
 'Fwd Packets/s',
 'Idle Max',
 'Max Packet Length',
 'Packet Length Mean',
 'Packet Length Variance',
 'SYN Flag Count',
 'Subflow Bwd Bytes',
 'Subflow Bwd Packets',
 'Subflow Fwd Bytes',
 'Subflow Fwd Packets',
 'Total Backward Packets',
 'Total Fwd Packets'}

In [72]:
df = df.drop(columns=list(features_to_drop))

### 남아있는 필드명 확인

In [73]:
columns_list = df.columns.tolist()
columns_list

['Destination Port',
 'Flow Duration',
 'Total Length of Fwd Packets',
 'Total Length of Bwd Packets',
 'Fwd Packet Length Min',
 'Fwd Packet Length Mean',
 'Fwd Packet Length Std',
 'Bwd Packet Length Min',
 'Bwd Packet Length Std',
 'Flow Bytes/s',
 'Flow Packets/s',
 'Flow IAT Mean',
 'Flow IAT Std',
 'Flow IAT Min',
 'Fwd IAT Mean',
 'Fwd IAT Std',
 'Fwd IAT Min',
 'Bwd IAT Total',
 'Bwd IAT Mean',
 'Bwd IAT Std',
 'Bwd IAT Max',
 'Bwd IAT Min',
 'Fwd PSH Flags',
 'Fwd Header Length',
 'Bwd Packets/s',
 'Min Packet Length',
 'Packet Length Std',
 'FIN Flag Count',
 'RST Flag Count',
 'PSH Flag Count',
 'ACK Flag Count',
 'URG Flag Count',
 'Down/Up Ratio',
 'Init_Win_bytes_forward',
 'Init_Win_bytes_backward',
 'act_data_pkt_fwd',
 'min_seg_size_forward',
 'Active Mean',
 'Active Std',
 'Active Max',
 'Idle Mean',
 'Idle Std',
 'Idle Min',
 'Label_binary']

## 결측치 처리
- 수치형: 평균
- 범주형: 최빈값

In [74]:
df = df.replace([np.inf, -np.inf], np.nan)  # inf 값을 NaN으로 변환

for col in df.columns:
    if df[col].dtype == 'object':
        df[col].fillna(df[col].mode()[0], inplace=True)
    else:
        df[col].fillna(df[col].mean(), inplace=True)

print(df.isnull().sum().sum())  # 전체 NaN 개수
print(np.isinf(df.select_dtypes(include=[np.number])).sum().sum())  # 전체 inf 개수

0
0


C:\Users\Admin\AppData\Local\Temp\ipykernel_23464\3983051279.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


## 이상치 처리

In [75]:
# 수치형 컬럼만 선택 (정규화/라벨 컬럼 등은 제외)
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
numerical_cols = [col for col in numerical_cols if col not in ['Label', 'Label_binary']]

# IQR 계산 및 클리핑
Q1 = df[numerical_cols].quantile(0.25)
Q3 = df[numerical_cols].quantile(0.75)
IQR = Q3 - Q1

# 각 feature별로 lower, upper 구해서 클리핑
for col in numerical_cols:
    lower = Q1[col] - 1.5 * IQR[col]
    upper = Q3[col] + 1.5 * IQR[col]
    df[col] = df[col].clip(lower=lower, upper=upper)

# 이상치 처리 결과 확인
df[numerical_cols].describe()

,Destination Port,Flow Duration,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Min,Bwd Packet Length Std,Flow Bytes/s,...,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Idle Mean,Idle Std,Idle Min
count,283074.000000,2.830740e+05,283074.000000,283074.000000,283074.000000,283074.000000,283074.000000,283074.000000,283074.000000,283074.000000,...,283074.000000,283074.000000,283074.000000,283074.0,283074.000000,283074.0,283074.000000,2.830740e+05,283074.0,2.830740e+05
mean,269.424633,5.033961e+06,57.596353,4064.836347,5.117058,26.093659,6.866872,3.888821,993.076890,21465.981536,...,3655.500857,128.062514,2.596344,20.0,642.240061,0.0,642.245519,3.959902e+06,0.0,3.116726e+06
std,351.665841,7.159623e+06,49.884403,5896.776450,5.898071,29.175139,9.132864,6.122912,1581.138824,31534.719341,...,5406.946375,155.617096,2.260781,0.0,1040.459886,0.0,1040.458716,6.540379e+06,0.0,5.003648e+06
min,0.000000,-1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-46809.969232,...,-1.000000,-1.000000,0.000000,20.0,0.000000,0.0,0.000000,0.000000e+00,0.0,0.000000e+00
25%,80.000000,3.301200e+04,26.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,21.314255,...,229.000000,-1.000000,1.000000,20.0,0.000000,0.0,0.000000,0.000000e+00,0.0,0.000000e+00
50%,80.000000,9.102870e+05,30.000000,154.000000,6.000000,8.666667,0.000000,0.000000,0.000000,2630.548131,...,256.000000,0.000000,2.000000,20.0,0.000000,0.0,0.000000,0.000000e+00,0.0,0.000000e+00
75%,443.000000,7.971701e+06,76.000000,11601.000000,6.000000,41.000000,10.263203,6.000000,2177.344966,31242.169912,...,8192.000000,229.000000,4.000000,20.0,1011.000000,0.0,1011.000000,7.501646e+06,0.0,6.343558e+06
max,987.500000,1.987973e+07,151.000000,29002.500000,15.000000,93.500000,25.658007,15.000000,5443.362415,78073.453399,...,20136.500000,574.000000,8.500000,20.0,2527.500000,0.0,2527.500000,1.875412e+07,0.0,1.585889e+07


## 수치형 정규화
- 수치형 데이터를 0~1 범위로 스케일링

In [76]:
# 수치형 컬럼 추출 (라벨/정답 컬럼은 제외)
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
numerical_cols = [col for col in numerical_cols if col not in ['Label', 'Label_binary']]

# 정규화 객체 생성 및 fit_transform
scaler = MinMaxScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# 정규화 결과 확인
df[numerical_cols].head()

,Destination Port,Flow Duration,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Min,Bwd Packet Length Std,Flow Bytes/s,...,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Idle Mean,Idle Std,Idle Min
0,1.0,2.012099e-07,0.079470,0.000000,0.4,0.064171,0.0,0.0,0.0,1.0,...,0.001688,0.000000,0.117647,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,5.533273e-06,0.039735,0.000207,0.4,0.064171,0.0,0.4,0.0,1.0,...,0.001490,0.446957,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,2.666031e-06,0.039735,0.000207,0.4,0.064171,0.0,0.4,0.0,1.0,...,0.001490,0.446957,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.760587e-06,0.039735,0.000207,0.4,0.064171,0.0,0.4,0.0,1.0,...,0.001589,0.573913,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,2.012099e-07,0.079470,0.000000,0.4,0.064171,0.0,0.0,0.0,1.0,...,0.001639,0.000000,0.117647,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 결과 저장

In [77]:
save_path = f'../data/preprocessed/{filename}'
df.to_csv(save_path, index=False)